In [2]:
import torch
from torch import nn
from mmcv.cnn.bricks.conv_module import ConvModule

In [14]:
class BasicBlock(nn.Module):
    
    def __init__(self,in_channels,
                 out_channels,
                kernel=3,
                 stride=1,
                 pad=1,
                 dilation=1,
                 con_cfg=None,
                 act_cfg=None,
                 norm_cfg=None,
                ):
        super().__init__()
        self.conv1 =ConvModule(in_channels,out_channels,kernel,stride,pad,dilation=dilation,
                              conv_cfg=con_cfg,act_cfg=act_cfg,norm_cfg=norm_cfg)
        self.conv2 = ConvModule(out_channels,out_channels,kernel,1,0,dilation=dilation,
                              conv_cfg=con_cfg,act_cfg=act_cfg,norm_cfg=norm_cfg)
    
    def forward(self,x):
        residual =x
        out =self.conv1(x)
        out = self.conv2(out)
        return out+residual
class DRNlayer(nn.Module):
    def __init__(self,
                 down_sample=False,
                num_layer=2,
                 stride=1,
                 **kwargs):
        super().__init__(**kwargs)
        self.layer =nn.ModuleList()
        if down_sample:
            self.layer.append(BasicBlock(stride=2,**kwargs))
        else:
            self.layer.append(BasicBlock(stride=1,**kwargs))
        if num_layer>1:
            self.layer.append(BasicBlock(stride=1,**kwargs))
    def forward(self,x):
        for layer in self.layer:
            out = layer(x)
        return out

In [11]:
class DRN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.stem = nn.Sequential(nn.Conv2d(3,16,7,5,3), BasicBlock(16,16,stride=2))
        self.block1 = BasicBlock() 
    def forward(self,x):
        out =self.stem(x)
        
    

In [12]:
drn = DRN()
x= torch.randn(1,3,224,224)
drn.eval()
out= drn(x)
print(out.shape)

torch.Size([1, 16, 45, 45])
